## Modèles par points : surfacique

####  Noms prénoms : ZHANI Reda , YAHYA Zakaria
#### Groupe : B2
#### Numéro du groupe : 22

#### Import

In [ ]:
#! pip install trimesh

In [2]:
import numpy as np
import trimesh
import plotly.graph_objs as go
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import random

#### Collecte des données

In [3]:
def nuage_de_points(path_to_model):
    """
    args:path_to_model: Chemin vers le fichier OBJ du modèle 3D.
    
    return:sommets: les nuages de points
    """
    # Charger le modèle avec Trimesh
    modele = trimesh.load(path_to_model, force='mesh')
    ## Coordonnées de sommets qui represente les nuages de points : np.array nx3
    sommets = np.array(modele.vertices)

    return sommets


'''
Il faut changer le chemin selon emplacement du dossier models3D
'''
#path_models3D = "/path/to/folder/"
path_models3D = "/home/n7student/Bureau/hakim/etudiant/models3D/"

## le modéle 3d chosie 
nom_mesh = "sphere" 
## Chargement des nuages de points
nuage_pts = nuage_de_points(path_models3D+nom_mesh+".obj")

#### Affichage des nuages de points

In [4]:
def tracer_nuages_pts(nuage_pts):
    x, y, z = nuage_pts[:, 0], nuage_pts[:, 1], nuage_pts[:, 2]

    # Tracer les nuages de points en bleu
    trace = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=4,
            color='blue',
            opacity=0.5
        )
    )
    data = [trace]
    
    # Créer la mise en page de la figure
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        ),
        width=800,
        height=800
    )

    # Créer et afficher la figure
    fig = go.Figure(data=data, layout=layout)
    fig.show()

tracer_nuages_pts(nuage_pts)


#### Calculs des normales en chaque point

In [5]:
def compute_direct_neighbors(points, k):
    
    """
    Utiliser NearestNeighbors de scikit-learn pour trouver les voisins les plus proches

    args: points: les nuages de points.
          k : Le nombre de voisins.

    return:neighbors : Un dictionnaire où chaque clé est l'indice d'un point et la valeur correspondante
                       est une liste d'indices des voisins directs de ce point.
    """

    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(points)
    distances, indices = nbrs.kneighbors(points)
    
    neighbors = {}
    for i, indices_i in enumerate(indices):
        # Exclure le point lui-même de la liste des voisins
        neighbors[i] = list(indices_i[indices_i != i])
    
    return neighbors

In [6]:
def compute_normals(points, neighbors):

    """
    calculer les normales pour chaque point.

    arg: points: nuages de points.
         neighbors : resultat de compute_direct_neighbors.

    return: normals : une liste contenant les normales calculées pour chaque point.
    """
    
    normals = []

    for i, point in enumerate(points):
        # Sélectionner les indices des voisins du point actuel
        neighbor_indices = neighbors[i]
        
        # Sélectionner les coordonnées des voisins du point actuel
        neighbor_points = points[neighbor_indices]
        
        # Centrer les données
        centered_points = neighbor_points - np.mean(neighbor_points, axis=0)
        
        # Calculer la matrice de covariance
        covariance_matrix = np.cov(centered_points, rowvar=False)
        
        # Calculer les valeurs propres et les vecteurs propres
        eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
        
        # Sélectionner le vecteur propre correspondant à la plus petite valeur propre comme estimation de la normale
        normal = eigenvectors[:, np.argmin(eigenvalues)]
        # Vérifier et corriger l'orientation de la normale
        if np.dot(normal, point) < 0 :
            normal = -normal
    
        normals.append(normal)
    
    return np.array(normals)


#### Affichage des normales

In [7]:
k = 50  # Nombre de voisins à considerer
neighbors = compute_direct_neighbors(nuage_pts, k)
normals = compute_normals(nuage_pts, neighbors)

In [8]:
def tracer_normales(nuage_pts, normals):

    x, y, z = nuage_pts[:, 0], nuage_pts[:, 1], nuage_pts[:, 2]
    # Tracer les nuages de points en bleu
    trace_points = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=4,
            color='blue',
            opacity=0.5
        ),
        name='Points'
    )

    # Tracer les normales en chaque points avec des cones
    normals_length = 0.01 # Longueur des vecteurs représentant les normales
    normals_trace = go.Cone(
        x=x,
        y=y,
        z=z,
        u=normals[:,0] * normals_length,  # Direction x des vecteurs
        v=normals[:,1] * normals_length,
        w=normals[:,2] * normals_length,
        colorscale='Viridis',              
        sizemode='absolute',               
        sizeref=0.5,                       
        showscale=False,                  
        name='Normals'                     
    )

    data = [trace_points, normals_trace]

    # Créer la mise en page de la figure
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        ),
        width=800,
        height=800
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()


tracer_normales(nuage_pts, normals)


#### Calcul des rayons en chaque point

In [9]:
def make_pca(data: np.ndarray, origin: np.ndarray, n_components: int):
    """
    Compute the PCA on the data provided
    :param n_components: n_components for the PCA
    :param data: Array containing data
    :return: projection of data 
    """
    x = np.vstack((np.array(data), origin))
    xc = x - origin
    pca = PCA(n_components=n_components)
    pca.fit(xc)
    w = pca.components_
    new_coord = xc @ np.transpose(w)
    return new_coord

def lstsq_quadrics_fitting(pos_xyz):
    """
    Fit a given set of 3D points (x, y, z) to a quadrics of equation ax^2 + by^2 + cxy + dx + ey + f = z
    :param pos_xyz: A two-dimensional numpy array, containing the coordinates of the points
    :return:
    """
    ## Quadric equation
    A = np.ones((pos_xyz.shape[0], 6))
    A[:, 0:2] = np.square(pos_xyz[:, 0:2], pos_xyz[:, 0:2])
    A[:, 2] = np.multiply(pos_xyz[:, 0], pos_xyz[:, 1])
    A[:, 3:5] = pos_xyz[:, 0:2]
    ## Z-axis
    Z = pos_xyz[:, 2]
    ## coefficents of the quatrics equation
    X, _, _, _ = np.linalg.lstsq(A, Z, rcond=None)
    
    return X

def curvature_meynet(quadrics):
    a = quadrics[0]
    b = quadrics[1]
    c = quadrics[2]
    d = quadrics[3]
    e = quadrics[4]
    f = quadrics[5]
    
    return ((1+d**2)*a+(1+e**2)*b-4*a*b*c)/(1+e**2+d**2)**(3/2)


def compute_rayons(data, neighbors):

    """
    calculer les rayons pour chaque point.

    arg: points : nuages de points.
         neighbors: resultat de compute_direct_neighbors.

    return: rayons : une liste contenant les rayons calculées pour chaque point.
    """
    
    rayons = []
    
    for i, point in enumerate(data):
        # Sélectionner les indices des voisins du point actuel
        neighbor_indices = neighbors[i]
        
        # Sélectionner les coordonnées des voisins du point actuel
        neighbor_points = data[neighbor_indices]
        
        new_coord = make_pca(neighbor_points, point,3)
        ## Calculs des coefficents de la meilleure quadric
        X = lstsq_quadrics_fitting(new_coord)
        
        ## rayon associées au point i 
        courbure = curvature_meynet(X)
        if courbure == 0 : 
            # Si la courbure est 0 alors le rayon de convergence est infini
            rayon = 10 
        else : 
            rayon = 1/courbure
        
        rayons.append(np.abs(rayon))
    
    return np.array(rayons)

In [10]:
rayons = compute_rayons(nuage_pts,neighbors)

#### Tracer une sphère tangente à un point en utilisant le rayon de courbure et la normale en ce point

In [11]:
def tracer_sphere_tangent(point, normal, rayon, data_points,neighbors):

    '''
    Tracer une sphère tangente à point en utilisant le rayon de courbure et la normale en ce point. 
    '''

    # Déplacer le centre de la sphère le long de la normale à une distance égale au rayon 
    center = point - rayon * normal

    # Générer les coordonnées des points sur la sphère
    theta = np.linspace(0, 2 * np.pi, 100)
    phi = np.linspace(0, np.pi, 100)
    x = center[0] + rayon * np.outer(np.cos(theta), np.sin(phi))
    y = center[1] + rayon * np.outer(np.sin(theta), np.sin(phi))
    z = center[2] + rayon * np.outer(np.ones(np.size(theta)), np.cos(phi))

    # Tracer la sphère
    fig = go.Figure(data=[go.Surface(x=x, y=y, z=z, opacity=0.5, showscale=False)])

    # Ajouter le point sur la surface
    fig.add_trace(go.Scatter3d(x=[point[0]], y=[point[1]], z=[point[2]], mode='markers', marker=dict(color='red', size=5)))
    for pt_i in neighbors : 
            surface_point = data_points[pt_i,:]
            fig.add_trace(go.Scatter3d(x=[point[0]], y=[point[1]], z=[point[2]], mode='markers', marker=dict(color='red', size=5)))
    # Ajouter les points du nuage
    fig.add_trace(go.Scatter3d(x=data_points[:,0], y=data_points[:,1], z=data_points[:,2], mode='markers', marker=dict(color='blue', size=3)))

    # Mettre en forme la visualisation
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        ),
        width=800, 
        height=800,
        showlegend=False
        
    )
    fig.show()
    
i = 69
tracer_sphere_tangent(nuage_pts[i],normals[i],rayons[i],nuage_pts ,neighbors[i])


#### Recherche des points de la surface definie par les nuages de points

In [12]:
def rbf_interp(x, y, z, sigma):

    """
    réaliser l'interpolation des données en utilisant la méthode de la fonction de base radiale (RBF).

    arg: x , y , z : Coordonnées x , y , z des points à interpoler.
        sigma : Paramètre pour la fonction RBF.

    return: weights : Poids obtenus à partir de RBF.
    """
    
    n = len(x)
    rbf_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            rbf_matrix[i, j] = np.exp(-sigma * np.linalg.norm([x[i]-x[j], y[i]-y[j]]))

    # Résolution du système linéaire rbf_matrix * weights = z pour obtenir les poids
    weights = np.linalg.solve(rbf_matrix, z)
    return weights

def rbf_surface(x_range, y_range, points, sigma):

    """
    Cette fonction génère une surface interpolée à partir des données en utilisant RBF.

    arg: x_range (tuple): Plage des valeurs x pour la surface interpolée.
         y_range (tuple): Plage des valeurs y pour la surface interpolée.
         points : Nuage de points à interpoler.

    return: X : Coordonnées x de la grille de la surface interpolée.
            Y : Coordonnées y de la grille de la surface interpolée.
            Z : Valeurs z de la surface interpolée.
    """
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]

    X, Y = np.meshgrid(np.linspace(x_range[0], x_range[1], 100), np.linspace(y_range[0], y_range[1], 100))
    Z = np.zeros_like(X)

    weights = rbf_interp(x, y, z, sigma)

    for i in range(len(X)):
        for j in range(len(X[0])):
            Z[i, j] = np.sum([weights[k] * np.exp(-sigma * np.linalg.norm([X[i, j]-x[k], Y[i, j]-y[k]])) for k in range(len(x))])

    return X, Y, Z

In [13]:
def tracer_rbf_surface(i, sigma, points, neighbors,N):
    fig = go.Figure()

    # Tracer les nuages points en bleu
    fig.add_trace(go.Scatter3d(x=points[:, 0], y=points[:, 1], z=points[:, 2], mode='markers', marker=dict(color='blue'), name='Les nuages de points'))

    # les indices des points voisins
    neighbor_indices = neighbors[i]

    # Tracer les voisins en rouge
    fig.add_trace(go.Scatter3d(x=points[neighbor_indices, 0], y=points[neighbor_indices, 1], z=points[neighbor_indices, 2], mode='markers', marker=dict(color='red'), name='Neighbors'))

    # Calculer la surface interpolante
    neighbor_points = points[neighbor_indices]
    x_range = (np.min(neighbor_points[:, 0]), np.max(neighbor_points[:, 0]))
    y_range = (np.min(neighbor_points[:, 1]), np.max(neighbor_points[:, 1]))

    X_interpolating, Y_interpolating, Z_interpolating = rbf_surface(x_range, y_range, neighbor_points, sigma)
    
    # Tracer N points interpolés en jaune
    random_indices = random.sample(range(len(X_interpolating.flatten())), N)
    X_interpolating_selected = X_interpolating.flatten()[random_indices]
    Y_interpolating_selected = Y_interpolating.flatten()[random_indices]
    Z_interpolating_selected = Z_interpolating.flatten()[random_indices]
    fig.add_trace(go.Scatter3d(x=X_interpolating_selected, y=Y_interpolating_selected, z=Z_interpolating_selected, mode='markers', marker=dict(color='yellow'), name='Points trouvés'))

    fig.add_trace(go.Surface(x=X_interpolating, y=Y_interpolating, z=Z_interpolating, colorscale='Earth', name='Interpolating Surface',showscale=False))

    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='cube'), width=800, height=800)

    fig.show()


#### Affichage de  Tracer la surface locale et les N points interpolés en jaune

In [14]:
tracer_rbf_surface(i=1, sigma=1.0, points=nuage_pts, neighbors=neighbors,N=7)